<div align='center'>
<font size="5"><strong>Wikipedia Pageviews Project</strong></font>
<br />
<i>Netanel Madmoni</i>
<br />
<strong><i><large>Part 1: Data Description, Cleaning and Transformation</large></i></strong>
</div>


----------------

# Data Description

1. Get List of people on Wikipedia
```sql
SELECT DISTINCT en_wiki -- page title name in english wikipedia	
FROM  `project.wikipedia_pageviews.wikidata`,
      UNNEST(instance_of) AS instance_of_struct

    
WHERE instance_of_struct.numeric_id = 5 -- instance_of = 5 => person
```

2. Get pageview data for those people

      ```sql
      SELECT title, DATETIME_TRUNC(datehour, MONTH) AS month, SUM(views) AS monthly_views

                  
      FROM  `project.wikipedia_pageviews.pageviews_2023` a
            JOIN `project.data_for_project.distinct_people` b
            ON a.title = b.en_wiki

      
      WHERE datehour IS NOT NULL
      AND wiki = "en"

      GROUP BY title, DATETIME_TRUNC(datehour, MONTH)
      ```

3. Get wikidata for those people
      ```sql
      SELECT *	
      FROM  `rising-theater-416315.wikipedia_pageviews.wikidata`,
      UNNEST(instance_of) AS instance_of_struct

    
      WHERE instance_of_struct.numeric_id = 5
      ```

# Data Cleaning & Transformation

In [1]:
#imports
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import json
import seaborn as sns
from rich import print
import time

load_dotenv()

RAW_DATA_DIR = Path(os.getenv('RAW_DATA_DIR'))
PROCESSED_DATA_DIR = Path(os.getenv('PROCESSED_DATA_DIR'))

## Views Data

Are organized in csv files, one per year.

In [3]:
# Read raw data
dfs = []
for file in RAW_DATA_DIR.glob(r'monthly_views_*.csv'):
    print(f'{file.name} - {os.stat(file).st_size / 1024 ** 2:.2f} MB')
    dfs.append(pl.read_csv(file))

print(f'Total files: {len(dfs)}')

monthly_views_2015.csv - 411.06 MB)

In [37]:
try:
    views_df = pl.read_parquet(PROCESSED_DATA_DIR / 'pageviews_all.parquet')
except FileNotFoundError:
    # Pivot
    transformed_dfs = []
    for df in tqdm(dfs):
        transformed_df = (df.with_columns(pl.col('month')
                                        .map_elements(lambda s: f'{s[:4]}-{s[5:7]}')
                                        .alias('year_month'))
                        .pivot(index='title', columns='year_month', values='monthly_views')
        )
        transformed_dfs.append(transformed_df)
    # Join
    all_names = pl.concat([df.select('title') for df in dfs]).unique()
    print(f'{len(all_names):,}')
    views_df = all_names
    for i, df in tqdm(enumerate(transformed_dfs)):
        views_df = df.join(views_df, 'title', 'outer_coalesce')
    views_df = views_df.select(['title', *sorted([c for c in views_df.columns if c != 'title'])])
    views_df.write_parquet(f'{PROCESSED_DATA_DIR}/pageviews_all.parquet')

views_df

title,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,…,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Neophyte_II_of…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,14,17,26,10,17,24,29,32,51,20,36,46,32,119,27,32,18,26,22,30,30,26,35,33,33,26,25,24,29,44,23,25
"""Olivia_Giovett…",26,34,52,20,41,40,19,21,27,23,32,21,32,29,57,47,31,21,24,15,42,17,45,30,25,34,36,22,22,22,16,28,30,34,22,19,…,39,30,32,34,37,29,14,26,24,36,39,35,20,63,33,39,19,19,30,20,14,21,35,52,25,25,33,20,18,22,31,26,16,20,59,41,19
"""Alexander_Gran…",9,48,24,23,32,24,12,20,19,7,8,14,18,27,11,23,12,19,15,17,11,13,26,14,16,18,13,20,13,26,16,19,32,14,18,12,…,5,13,9,12,11,9,14,19,9,7,5,11,4,11,9,7,9,9,4,5,12,72,6,10,10,10,2,7,10,7,2,7,4,9,10,1,5
"""Bernard_H._Rae…",17,16,21,20,19,18,10,9,17,5,8,10,5,28,19,11,17,14,13,19,16,13,24,18,10,10,14,13,14,19,5,15,26,12,13,10,…,2,1,4,1,5,5,3,5,2,2,3,4,null,12,4,3,2,1,4,null,1,3,8,6,3,1,5,1,3,5,7,3,1,2,null,1,2
"""Leigh_Magar""",47,46,64,36,56,45,41,26,30,48,15,26,19,38,28,38,37,51,24,36,17,15,45,39,26,22,21,14,19,23,16,32,36,21,18,20,…,8,8,9,13,6,3,12,13,12,5,8,8,11,3,9,8,13,13,9,20,9,8,14,27,14,13,20,13,10,9,5,9,9,8,5,10,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Edgar_Dibden""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,16,6,10,6,7,4,2
"""David_Mulready…",56,51,78,51,54,59,36,44,31,33,29,29,29,54,44,45,23,36,38,35,32,31,42,27,44,33,27,52,29,41,21,36,47,64,37,30,…,16,21,27,33,15,22,31,15,25,16,25,12,25,27,31,18,27,25,20,21,34,15,16,14,15,19,26,19,18,12,7,19,12,14,14,7,6
"""Bruno_Riem""",40,29,34,17,31,29,22,31,14,8,8,16,13,16,7,17,24,16,20,14,14,7,23,16,25,14,13,15,13,20,10,17,31,14,13,12,…,4,9,2,4,5,4,7,8,6,7,8,5,5,4,4,7,2,5,4,11,2,4,5,5,3,5,2,2,4,3,7,5,4,3,3,1,2


## Verified People Data

Data from https://data.sciencespo.fr/dataset.xhtml?persistentId=doi:10.21410/7E4/RDAG3O.

In [34]:
people_data = pl.read_csv(RAW_DATA_DIR / 'cross-verified-database.csv',
                          ignore_errors=True,
                          columns=['wikidata_code', 'name',
                                   'birth', 'death', 'bigperiod_birth', 'bigperiod_death',
                                   'un_subregion', 'un_region',
                                   'citizenship_1_b',
                                   'bplo1', 'bpla1', 'dplo1', 'dpla1',
                                   'gender',
                                   'level1_main_occ', 'level2_main_occ', 'level3_main_occ',
                                   'sum_visib_ln_5criteria', 'ranking_visib_5criteria'])
people_data

wikidata_code,birth,death,gender,level1_main_occ,name,un_subregion,level2_main_occ,level3_main_occ,bigperiod_birth,bigperiod_death,sum_visib_ln_5criteria,ranking_visib_5criteria,citizenship_1_b,un_region,bplo1,dplo1,bpla1,dpla1
str,i64,i64,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,f64,f64,f64
"""Q1000002""",1932,1990,"""Male""","""Culture""","""Claus_Hammel""","""Western Europe…","""Culture-core""","""playwright""","""5.Contemporary…","""5.Contemporary…",18.083672,1.058542e6,"""Germany""","""Europe""",11.833333,12.42,53.416668,54.38139
"""Q1000005""",1860,1927,"""Male""","""Culture""","""Karel_Matěj_Ča…","""Western Europe…","""Culture-core""","""writer""","""4.Mid Modern P…","""5.Contemporary…",23.98061,131428.0,"""Czech_Republic…","""Europe""",12.929798,14.421389,49.440605,50.087502
"""Q1000006""",1971,null,"""Male""","""Culture""","""Florian_Eichin…","""Western Europe…","""Culture-core""","""film""","""5.Contemporary…","""Missing""",20.666656,775768.0,"""Germany""","""Europe""",9.191944,null,48.897499,null
"""Q1000015""",1983,null,"""Male""","""Culture""","""Florian_Jahr""","""Western Europe…","""Culture-core""","""actor""","""5.Contemporary…","""Missing""",21.18504,691735.0,"""Germany""","""Europe""",13.383333,null,52.516666,null
"""Q1000023""",1912,1977,"""Female""","""Leadership""","""Wiltraut_Rupp-…","""Western Europe…","""Administration…","""judge""","""5.Contemporary…","""5.Contemporary…",17.99621,1.103282e6,"""Germany""","""Europe""",13.35,8.4,52.4333,49.016666
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Q999994""",1988,null,"""Male""","""Sports/Games""","""Mitja_Mežnar""","""Southern Europ…","""Sports/Games""","""ski""","""5.Contemporary…","""Missing""",21.543238,350041.5,"""Slovenia""","""Europe""",14.35561,null,46.238869,null
"""Q999995""",1987,null,"""Male""","""Sports/Games""","""Martin_Cikl""","""Western Europe…","""Sports/Games""","""ski""","""5.Contemporary…","""Missing""",21.71133,339942.5,"""Czech_Republic…","""Europe""",14.618354,null,50.911613,null
"""Q999997""",1984,null,"""Male""","""Sports/Games""","""Vincent_Descom…","""Western Europe…","""Sports/Games""","""ski""","""5.Contemporary…","""Missing""",24.453411,154890.0,"""France""","""Europe""",6.868889,null,45.922222,null


Columns description:

- `wikidata_code` - wikidata identifier of the entity
- `name` - page name
- `birth` - year of birth
- `death` - year of death
- `bigperiod_birth` - year of birth binned into historic periods
- `bigperiod_death` - year of death binned into historic periods
- `citizenship_1_b`, `un_region`, `un_subregion` - citizenship of the person in different levels
- `bplo1` - birth place longitude
- `bpla1` - birth place latitude
- `dplo1` - birth place longitude
- `dpla1` - birth place latituse
- `gender` - gender of the entity
- `level1_main_occ`, `level2_main_occ`, `level3_main_occ` - occupation of entity in different levels of specificity
- `sum_visib_ln_5criteria`, `ranking_visib_5criteria` - notability score and ranking (see written report for details).

## Entity Data

I would like to extract additional information from the wikidata dataset.

Are organized in json files.

In [2]:
info_files = (RAW_DATA_DIR / 'wikidata').glob('*')
len(list(info_files))

400

For example...

In [2]:
info_files = (RAW_DATA_DIR / 'wikidata').glob('*')
with open(next(info_files)) as f:
    for line in f:
        d = json.loads(line)
        sites = [sitelink['site'] for sitelink in d['sitelinks']]
        if 'enwiki' in sites:
            print(d)
            break
    

{
    'id': 'Q17122148',
    'numeric_id': '17122148',
    'en_label': 'Albrecht Joseph',
    'en_wiki': 'Albrecht_Joseph',
    'en_description': 'German screenwriter, film editor and author',
    'type': 'item',
    'sitelinks': [
        {'site': 'dewiki', 'title': 'Albrecht Joseph', 'encoded': 'Albrecht_Joseph'},
        {'site': 'enwiki', 'title': 'Albrecht Joseph', 'encoded': 'Albrecht_Joseph'},
        {'site': 'frwiki', 'title': 'Albrecht Joseph', 'encoded': 'Albrecht_Joseph'},
        {'site': 'arwiki', 'title': 'ألبرشت جوزيف', 'encoded': 'ألبرشت_جوزيف'},
        {'site': 'arzwiki', 'title': 'البرشت چوزيف', 'encoded': 'البرشت_چوزيف'},
        {'site': 'fawiki', 'title': 'آلبرشت یوزف', 'encoded': 'آلبرشت_یوزف'}
    ],
    'descriptions': [
        {'language': 'de', 'value': 'deutsch-amerikanischer Theater- und Filmschaffender'},
        {'language': 'fa', 'value': 'فیلمنامه\u200cنویس، تدوینگر، و نویسنده آلمانی'},
        {'language': 'en', 'value': 'German screenwriter, film editor and author'},
        {'language': 'nl', 'value': 'Duits scenarioschrijver (1901-1991)'},
        {'language': 'sv', 'value': 'tysk författare och filmklippare'}
    ],
    'labels': [
        {'language': 'de', 'value': 'Albrecht Joseph'},
        {'language': 'fr', 'value': 'Albrecht Joseph'},
        {'language': 'nl', 'value': 'Albrecht Joseph'},
        {'language': 'en', 'value': 'Albrecht Joseph'},
        {'language': 'es', 'value': 'Albrecht Joseph'},
        {'language': 'hu', 'value': 'Albrecht Joseph'},
        {'language': 'sl', 'value': 'Albrecht Joseph'},
        {'language': 'ca', 'value': 'Albrecht Joseph'},
        {'language': 'ast', 'value': 'Albrecht Joseph'},
        {'language': 'ar', 'value': 'ألبرشت جوزيف'},
        {'language': 'cs', 'value': 'Albrecht Joseph'},
        {'language': 'sk', 'value': 'Albrecht Joseph'},
        {'language': 'arz', 'value': 'البرشت چوزيف'},
        {'language': 'it', 'value': 'Albrecht Joseph'},
        {'language': 'sq', 'value': 'Albrecht Joseph'},
        {'language': 'he', 'value': "אלברכט ג'וזף"},
        {'language': 'sv', 'value': 'Albrecht Joseph'},
        {'language': 'ga', 'value': 'Albrecht Joseph'},
        {'language': 'fa', 'value': 'آلبرشت یوزف'},
        {'language': 'ru', 'value': 'Элбрехт Джозеф'}
    ],
    'aliases': [
        {'language': 'de', 'value': 'Al Joseph'},
        {'language': 'en', 'value': 'Al Joseph'},
        {'language': 'cs', 'value': 'Al Joseph'},
        {'language': 'sk', 'value': 'Al Joseph'}
    ],
    'instance_of': [{'numeric_id': '5'}],
    'gender': [{'numeric_id': '6581097'}],
    'date_of_birth': [{'time': '+1901-11-20T00:00:00Z'}],
    'date_of_death': [{'time': '+1901-11-20T00:00:00Z'}],
    'worked_at': [],
    'country_of_citizenship': [{'numeric_id': '183'}],
    'country': [],
    'educated_at': [],
    'occupation': [{'numeric_id': '28389'}, {'numeric_id': '7042855'}, {'numeric_id': '36180'}],
    'instrument': [],
    'genre': [],
    'industry': [],
    'subclass_of': [],
    'coordinate_location': [],
    'iso_3166_alpha3': [],
    'member_of': [],
    'from_fictional_universe': []
}

In [2]:
try:
    info_df = pl.read_parquet(PROCESSED_DATA_DIR / 'wikidata_all.parquet')
except FileNotFoundError:
    info_list = []
    info_files = (RAW_DATA_DIR / 'wikidata').glob('*')
    for file in info_files:
        with open(file) as f:
            for line in f:
                d = json.loads(line)
                sites = [sitelink['site'] for sitelink in d['sitelinks']]
                if 'enwiki' in sites:
                    info_list.append(d)
    info_df = pl.DataFrame(info_list)

with pl.Config(tbl_rows=3):
    display(info_df)

id,numeric_id,en_label,en_wiki,en_description,type,sitelinks,descriptions,labels,aliases,instance_of,gender,date_of_birth,date_of_death,worked_at,country_of_citizenship,country,educated_at,occupation,instrument,genre,industry,subclass_of,coordinate_location,iso_3166_alpha3,member_of,from_fictional_universe
str,str,str,str,str,str,list[struct[3]],list[struct[2]],list[struct[2]],list[struct[2]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[1]],list[struct[2]],list[null],list[struct[1]],list[struct[1]]
"""Q17122148""","""17122148""","""Albrecht Josep…","""Albrecht_Josep…","""German screenw…","""item""","[{""dewiki"",""Albrecht Joseph"",""Albrecht_Joseph""}, {""enwiki"",""Albrecht Joseph"",""Albrecht_Joseph""}, … {""fawiki"",""آلبرشت یوزف"",""آلبرشت_یوزف""}]","[{""de"",""deutsch-amerikanischer Theater- und Filmschaffender""}, {""fa"",""فیلمنامه‌نویس، تدوینگر، و نویسنده آلمانی""}, … {""sv"",""tysk författare och filmklippare""}]","[{""de"",""Albrecht Joseph""}, {""fr"",""Albrecht Joseph""}, … {""ru"",""Элбрехт Джозеф""}]","[{""de"",""Al Joseph""}, {""en"",""Al Joseph""}, … {""sk"",""Al Joseph""}]","[{""5""}]","[{""6581097""}]","[{""+1901-11-20T00:00:00Z""}]","[{""+1901-11-20T00:00:00Z""}]",[],"[{""183""}]",[],[],"[{""28389""}, {""7042855""}, {""36180""}]",[],[],[],[],[],[],[],[]
"""Q100707809""","""100707809""","""Robert Cahaly""","""Robert_Cahaly""","""American polls…","""item""","[{""enwiki"",""Robert Cahaly"",""Robert_Cahaly""}, {""jawiki"",""ロバート・カヘリー"",""ロバート・カヘリー""}]","[{""en"",""American pollster and political consultant""}, {""zh"",""美国民调专家""}]","[{""en"",""Robert Cahaly""}, {""de"",""Robert Cahaly""}, … {""pt-br"",""Robert Cahaly""}]",[],"[{""5""}]","[{""6581097""}]",[],[],"[{""100704905""}]","[{""30""}]",[],"[{""1024426""}]","[{""16919156""}, {""8125919""}]",[],[],[],[],[],[],[],[]
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Q6128908""","""6128908""","""James Ashurst""","""James_Ashurst""","""English minist…","""item""","[{""enwiki"",""James Ashurst"",""James_Ashurst""}]","[{""en"",""English minister""}, {""nl"",""ambtenaar""}]","[{""en"",""James Ashurst""}, {""es"",""James Ashurst""}, … {""sq"",""James Ashurst""}]",[],"[{""5""}]","[{""6581097""}]","[{""+1605-01-01T00:00:00Z""}]","[{""+1605-01-01T00:00:00Z""}]",[],[],[],[],"[{""212238""}]",[],[],[],[],[],[],[],[]


**Columns description** from the Wikidata Database (I've used the [Wikidata Property Explorer](https://prop-explorer.toolforge.org/)):
- `id`, `numeric_id` - entity identifiers.
- `en_wiki`, `en_label`, `en_description` - English wiki-name, label (human-readable name) and description of the entity.
- `type` - type of entity (item/property/etc. See https://www.wikidata.org/wiki/Wikidata:Identifiers).
- `sitelinks` - a list of *struct*s (dictionary-like data structure) containing site name, wiki-name, and label of the entity in all the wikis it is in.
- `descriptions`, `labels`, `aliases` - lists of structs containing site name, description of the entity in that site, label and aliases of entity in that site.
- `instance_of` - "that class[es] of which this subject is a particular example and member". In this dataset I've taken only entities that are an instance of `Human (Q5)`.
- `gender` - "sex or gender identity of human or animal. For human: male, female, non-binary, intersex, transgender female, transgender male, agender, etc."
- `date_of_birth`/`death` - "date on which the subject was born / died"
- `worked_at` - "location where persons or organisations were actively participating in employment, business or other work"
- `country_of_citizenship` - "the object is a country that recognizes the subject as its citizen"
- `country` - "sovereign state that this item is in ***(not to be used for human beings)***"
- `educated_at` - "educational institution attended by subject"
- `occupation` - "occupation of a person"
- `instrument` - "musical instrument that a person plays or teaches or used in a music occupation"
- `genre` - "creative work's genre or an artist's field of work"
- `industry` - "specific industry of company or organization"
- `subclass_of` - "this item is a subclass (subset) of that item; all instances of this item are instances of that item. different from P31 (instance of), e.g.: K2 is an instance of mountain; volcano is a subclass of mountain (and an instance of volcanic landform)"
- `coordinate_location` - "geocoordinates of the subject"
- `iso_3166_alpha3` - "identifier for a country in three-letter format"
- `member_of` - "organization, club or musical group to which the subject belongs"
- `from_fictional_universe` - "subject's fictional entity is in the object narrative".

### Cleaning the Data

Let's look at the columns and their data types.

In [3]:
print(dict(zip(info_df.columns, info_df.dtypes)))

{
    'id': String,
    'numeric_id': String,
    'en_label': String,
    'en_wiki': String,
    'en_description': String,
    'type': String,
    'sitelinks': List(Struct({'site': String, 'title': String, 'encoded': String})),
    'descriptions': List(Struct({'language': String, 'value': String})),
    'labels': List(Struct({'language': String, 'value': String})),
    'aliases': List(Struct({'language': String, 'value': String})),
    'instance_of': List(Struct({'numeric_id': String})),
    'gender': List(Struct({'numeric_id': String})),
    'date_of_birth': List(Struct({'time': String})),
    'date_of_death': List(Struct({'time': String})),
    'worked_at': List(Struct({'numeric_id': String})),
    'country_of_citizenship': List(Struct({'numeric_id': String})),
    'country': List(Struct({'numeric_id': String})),
    'educated_at': List(Struct({'numeric_id': String})),
    'occupation': List(Struct({'numeric_id': String})),
    'instrument': List(Struct({'numeric_id': String})),
    'genre': List(Struct({'numeric_id': String})),
    'industry': List(Struct({'numeric_id': String})),
    'subclass_of': List(Struct({'numeric_id': String})),
    'coordinate_location': List(Struct({'latitude': Float64, 'longitude': Float64})),
    'iso_3166_alpha3': List(Null),
    'member_of': List(Struct({'numeric_id': String})),
    'from_fictional_universe': List(Struct({'numeric_id': String}))
}

One of the challenges in this dataset is the abundance of `List` and `Struct` (a dictionary-like) data types. Meaning, each cell can have multiple values in it. This section will mostly focus on how to solve this problem.

✨ I shall treat the `List` columns differently than the "normal" columns when transforming and exploring the data.

Also, many columns are already present in the validated dataset.

* **Empty columns**

In [33]:
list_columns = [c for c in info_df.columns if info_df[c].dtype == pl.List]

empty = []

for column in [c for c in info_df.columns if c not in list_columns]:
    empty.append([column,
                         (info_df[column]
                         .is_null()
                         .value_counts()
                         .filter(pl.col(column))
                         .select('count')
                         .to_numpy()).squeeze()])

for column in list_columns:
    empty.append([column,
                         ((info_df[column]
                         .list.len() == 0)
                         .value_counts()
                         .filter(pl.col(column))
                         .select('count')
                         .to_numpy()).squeeze()])

for item in empty:
    if not item[1].any():
        item[1] = 0
    item.append(item[1] / info_df.height * 100)

empty_df = (pd.DataFrame(empty, columns=['column', 'num_empty', 'percent_empty'])
            .sort_values('percent_empty', ascending=False)
            )
empty_df.style.format({'percent_empty': '{:.3f}%', 'num_empty': '{:,}'})

,column,num_empty,percent_empty
24,iso_3166_alpha3,"1,993,459",100.000%
26,from_fictional_universe,"1,993,454",100.000%
22,subclass_of,"1,993,442",99.999%
23,coordinate_location,"1,993,434",99.999%
21,industry,"1,993,093",99.982%
16,country,"1,992,220",99.938%
20,genre,"1,918,168",96.223%
19,instrument,"1,904,085",95.517%
25,member_of,"1,882,253",94.421%
14,worked_at,"1,811,046",90.849%


✨ Column `iso_3166_alpha3` is completely empty. I shall drop it.

✨ There are a few columns that are >90% empty. I will leave them there for now, as they might prove useful later on.




* **Unique values**

In [37]:
unique_value_counts = []
for column in list_columns:
    unique_value_counts.append((column, info_df[column].list.explode().n_unique()))
for column in [c for c in info_df.columns if c not in list_columns]:
    unique_value_counts.append((column, info_df[column].n_unique()))
    
unique_df = (pd.DataFrame(unique_value_counts, columns=['column', 'unique_values'])
            .sort_values('unique_values')
            )
unique_df.style.format('{:,}', subset='unique_values')

,column,unique_values
26,type,1
18,iso_3166_alpha3,1
20,from_fictional_universe,6
16,subclass_of,19
17,coordinate_location,26
5,gender,45
15,industry,71
10,country,118
4,instance_of,357
13,instrument,767


✨ The column `type` contains only one value for all rows. I shall remove it.

Interesting points:
* apparently there are 45(!) different genders.
* Many columns have a lot of unique values.

✨ **Observations so far:**

There are a few types of columns:
1. Redundant columns: columns that don't add information, either because they contain the same value throughout, have many missing values or do not contribute to our specific needs. The columns are:
    - `instance_of` - uninformative (info included in the other columns)
    - `type` (same value for all rows)
    - `numeric_id` (contained in `id`)
    - `descriptions` (unneeded information)
    - `labels` (unneeded information)
    - `coordinate_location`(>99% empty)
    - `country` (according to the Wikidata site, this property should not be used for humans)
    - `date_of_death` (has wrong information)
    - `iso_3166_alpha3` (empty)
    - `member_of` (not interesting enough)
    - `sitelinks` (not interesting enough)
    - `aliases` (not interesting enough)
    - `industry` (included in other columns)
    - `subclass_of` (too few values)
    - `from_fictional_universe` (too few values)

2. Columns with a single value per row: columns that (should) have zero or one values for each row. These columns are:
    - `id`
    - `en_label`
    - `en_wiki`
    - `en_description`
    - `date of birth` (take only the first one for each row)

3. Columns with *'interesting'* values per row: columns that contain zero, one or more values for each row, where every (major) value is *interesting* for our purpose. These columns are:
    - `instrument`
    - `worked_at`
    - `educated_at`
    - `genre`

4. Columns that have multiple values per row: columns that have multiple values per row, where an aggregation of these values is *interesting* for our purposes. These columns are:
    - `gender`
    - `instrument`
    - `worked_at`
    - `country_of_citizenship`
    - `educated_at`
    - `occupation`
    - `genre`
    - `industry`

 **Note** that a column might be a member of both the third group as well as the fourth group.

Handling strategy for each type of column:
1. Redundant columns --> drop columns.
2. Columns with a single value per row: --> leave as is (flatten if in a list or a struct).
3. Columns with *'interesting'* values per row --> encode into binary columns (bin uncommon values, or combine with more common ones) and drop original.
4. Columns that have multiple values per row --> create a column with the aggregate metric and drop original

I will do parts 1, 2, and 4 now. Later I will transform the dataframe after looking at the value distributions (part 3).

In [29]:
def clean_info_df(df: pl.DataFrame):
      
      ## Drop columns, remove duplicate rows, unnest structs ########
      columns_to_drop = ['type', 'numeric_id', 'descriptions', 'labels',
                         'coordinate_location', 'country', 'member_of',
                         'iso_3166_alpha3', 'date_of_death', 'instance_of',
                         'sitelinks', 'aliases', 'industry', 'subclass_of', 'from_fictional_universe']
      struct_columns = [c for c in df.columns if df[c].dtype == pl.List and c not in columns_to_drop]

      df = (df
            .unique('id') # Remove duplicate rows
            .drop(columns_to_drop) # drop redundant columns
            .with_columns([pl.col(col).list.eval(pl.element().struct[0]) for col in struct_columns]) # unnest structs
            .with_columns(pl.col('date_of_birth').list[0].str.slice(6, 2).replace('00', None).alias('birth_month')) # Only the first date of birth
      )
      
      
      # Create columns based on aggregates (counts) ###############
      columns_to_aggregate = ['gender', 'instrument',
                              'worked_at', 'country_of_citizenship', 'educated_at',
                              'occupation', 'genre']
      df = df.with_columns([pl.col(col).list.len().alias(f'count_{col}') for col in columns_to_aggregate])
      df = df.drop(['country_of_citizenship', 'occupation', 'gender', 'date_of_birth'])
     
      return df

In [30]:
filepath = PROCESSED_DATA_DIR / 'wikidata_clean_new.parquet'

try:
    info_df_cleaned = pl.read_parquet(filepath)
except FileNotFoundError:
    info_df_cleaned = clean_info_df(info_df)
    info_df_cleaned.write_parquet(filepath)
    
info_df_cleaned

id,en_label,en_wiki,en_description,worked_at,educated_at,instrument,genre,birth_month,count_gender,count_instrument,count_worked_at,count_country_of_citizenship,count_educated_at,count_occupation,count_genre
str,str,str,str,list[str],list[str],list[str],list[str],str,u32,u32,u32,u32,u32,u32,u32
"""Q675282""","""Teodoro de Cro…","""Teodoro_de_Cro…","""knight (1730-1…",[],[],[],[],"""06""",1,0,0,1,0,1,0
"""Q2074431""","""Wilson Eduardo…","""Wilson_Eduardo…","""Portuguese foo…",[],[],[],[],"""07""",1,0,0,1,0,1,0
"""Q4725868""","""Alice Fulks""","""Alice_Fulks""","""American actre…",[],[],[],[],"""01""",1,0,0,1,0,1,0
"""Q1338017""","""Aleksandr Alme…","""Alexander_Alme…","""Russian ice ho…",[],[],[],[],"""01""",1,0,0,1,0,1,0
"""Q97482091""","""Prem Nababsing…","""Prem_Nababsing…","""Mauritian poli…",[],"[""2983886""]",[],[],"""01""",1,0,0,0,1,2,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Q6762382""","""Mariano Perfec…","""Mariano_Perfec…","""writer and Gov…",[],[],[],[],"""01""",1,0,0,0,0,1,0
"""Q4686431""","""Advent Bangun""","""Advent_Bangun""","""Indonesian kar…",[],[],[],[],"""10""",1,0,0,1,0,1,0
"""Q7177498""","""Peter Viggers""","""Peter_Viggers""","""British politi…",[],"[""7232466"", ""1244704"", ""9357897""]",[],[],"""03""",1,0,0,1,3,4,0


#### Converting Codes Into Labels

Many columns have 'numeric id's in them. These are numeric code to wikidata's property. To understand the data, we need to convert them to human-readable labels. I will grab them from the wikidata SPARQL endpoint.

First I'll get all the codes needed to be converted.

In [21]:
columns_with_codes = ['worked_at', 'educated_at', 'instrument', 'genre']
unique_codes = [info_df_cleaned[col].explode().unique().drop_nulls() for col in columns_with_codes]

In [31]:
([(df.name, len(df)) for df in unique_codes])

[('worked_at', 37768),
 ('educated_at', 46136),
 ('instrument', 766),
 ('genre', 3170)]

In [88]:
from SPARQLWrapper import SPARQLWrapper
from typing import Iterable
import re

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
regex = re.compile(r'(?<=\/Q)[0-9]*$')

def _convert_wikidata_codes_to_labels_single_chunk(codes: Iterable, wrapper: SPARQLWrapper, regex=regex):
    wrapper.setReturnFormat('json')
    formatted_codes = '\n'.join([f'wd:Q{code}' for code in codes])        
    query = '''
    SELECT ?wd ?wdLabel WHERE {
    VALUES ?wd {
    ''' \
    + formatted_codes + \
    '''
    }
    ?wd rdfs:label ?wdLabel.
    FILTER(LANG(?wdLabel) = 'en').
    }
    '''
    wrapper.setQuery(query)
    data = wrapper.query().convert()['results']['bindings']
    results = {regex.findall(r['wd']['value'])[0] : r['wdLabel']['value'] for r in data}
    return results

def convert_wikidata_codes_to_labels(codes: Iterable, chunksize: int = 350, wrapper: SPARQLWrapper = sparql):
    if len(codes) == 0:
        return
    if len(codes) <= chunksize:
        return _convert_wikidata_codes_to_labels_single_chunk(codes, wrapper)
    
    results = {}
    for offset in range(0, len(codes), chunksize):
        chunk = codes[offset:offset + chunksize]
        result = convert_wikidata_codes_to_labels(chunk, chunksize, wrapper)
        if result is None:
            break
        results |= result
    return results

In [92]:
t = tqdm(unique_codes)
for series in t:
    filepath = PROCESSED_DATA_DIR / 'codes_labels' / f'{series.name}.json'
    if filepath.exists():
        continue
    t.set_postfix_str(series.name)
    code2labels = convert_wikidata_codes_to_labels(series)
    time.sleep(1)
    with open(filepath, 'w') as f:
        json.dump(code2labels, f)

100%|██████████| 13/13 [06:10<00:00, 28.47s/it, from_fictional_universe]


In [18]:
codes_labels = {}
for file in (PROCESSED_DATA_DIR / 'codes_labels').glob('*.json'):
    with open(file) as f:
        codes_labels[file.name[:-5]] = json.load(f)

In [19]:
def decode_df(df: pl.DataFrame):
    return df.with_columns([
        pl.col(col).list.eval(pl.element().replace(codes_labels[col])) for col in columns_with_codes
    ])

In [32]:
filepath = PROCESSED_DATA_DIR / 'wikidata_decoded_new.parquet'
try:
    info_df_decoded = pl.read_parquet(filepath)
except FileNotFoundError:
    info_df_decoded = decode_df(info_df_cleaned)
    info_df_decoded.write_parquet(filepath)

info_df_decoded

id,en_label,en_wiki,en_description,worked_at,educated_at,instrument,genre,birth_month,count_gender,count_instrument,count_worked_at,count_country_of_citizenship,count_educated_at,count_occupation,count_genre
str,str,str,str,list[str],list[str],list[str],list[str],str,u32,u32,u32,u32,u32,u32,u32
"""Q675282""","""Teodoro de Cro…","""Teodoro_de_Cro…","""knight (1730-1…",[],[],[],[],"""06""",1,0,0,1,0,1,0
"""Q2074431""","""Wilson Eduardo…","""Wilson_Eduardo…","""Portuguese foo…",[],[],[],[],"""07""",1,0,0,1,0,1,0
"""Q4725868""","""Alice Fulks""","""Alice_Fulks""","""American actre…",[],[],[],[],"""01""",1,0,0,1,0,1,0
"""Q1338017""","""Aleksandr Alme…","""Alexander_Alme…","""Russian ice ho…",[],[],[],[],"""01""",1,0,0,1,0,1,0
"""Q97482091""","""Prem Nababsing…","""Prem_Nababsing…","""Mauritian poli…",[],"[""Royal College Curepipe""]",[],[],"""01""",1,0,0,0,1,2,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Q6762382""","""Mariano Perfec…","""Mariano_Perfec…","""writer and Gov…",[],[],[],[],"""01""",1,0,0,0,0,1,0
"""Q4686431""","""Advent Bangun""","""Advent_Bangun""","""Indonesian kar…",[],[],[],[],"""10""",1,0,0,1,0,1,0
"""Q7177498""","""Peter Viggers""","""Peter_Viggers""","""British politi…",[],"[""Portsmouth Grammar School"", ""Trinity Hall"", ""University of Law""]",[],[],"""03""",1,0,0,1,3,4,0


Finally, I will join the two info tables and filter only the values present in the views dataframe.

In [43]:
filepath = PROCESSED_DATA_DIR / 'people_data_all.parquet'
try:
    people_data_all = pl.read_parquet(filepath)
except FileNotFoundError:
    people_data_all = people_data.join(info_df_decoded.unique('en_label'), how='left', left_on='wikidata_code', right_on='id')
    people_data_all = people_data_all.filter(pl.col('name').is_in(views_df['title']).or_(pl.col('en_wiki').is_in(views_df['title']))).drop('en_wiki')
    people_data_all.write_parquet(filepath)
people_data_all

wikidata_code,birth,death,gender,level1_main_occ,name,un_subregion,level2_main_occ,level3_main_occ,bigperiod_birth,bigperiod_death,sum_visib_ln_5criteria,ranking_visib_5criteria,citizenship_1_b,un_region,bplo1,dplo1,bpla1,dpla1,en_label,en_description,worked_at,educated_at,instrument,genre,birth_month,count_gender,count_instrument,count_worked_at,count_country_of_citizenship,count_educated_at,count_occupation,count_genre
str,i64,i64,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,f64,f64,f64,str,str,list[str],list[str],list[str],list[str],str,u32,u32,u32,u32,u32,u32,u32
"""Q1000005""",1860,1927,"""Male""","""Culture""","""Karel_Matěj_Ča…","""Western Europe…","""Culture-core""","""writer""","""4.Mid Modern P…","""5.Contemporary…",23.98061,131428.0,"""Czech_Republic…","""Europe""",12.929798,14.421389,49.440605,50.087502,"""Karel Matěj Ča…","""Czech writer a…",[],[],[],[],"""02""",1,0,0,1,0,4,0
"""Q100005""",1922,1951,"""Male""","""Culture""","""Tadeusz_Borows…","""Eastern Europe…","""Culture-core""","""writer""","""5.Contemporary…","""5.Contemporary…",28.841227,21473.5,"""Poland""","""Europe""",28.657778,21.01111,50.254444,52.23,"""Tadeusz Borows…","""Polish writer …",[],"[""University of Warsaw""]",[],[],"""11""",1,0,0,1,1,5,0
"""Q1000051""",1884,1962,"""Male""","""Leadership""","""Joseph_C._O'Ma…","""Northern Ameri…","""Politics""","""politician""","""4.Mid Modern P…","""5.Contemporary…",23.15431,170792.0,"""US""","""America""",-71.033333,-77.094643,42.391666,38.984825,"""Joseph C. O'Ma…","""American journ…",[],"[""Georgetown University Law Center"", ""Columbia University""]",[],[],"""11""",1,0,0,1,2,3,0
"""Q1000053""",1962,null,"""Male""","""Leadership""","""Vasily_Nebenzy…","""Eastern Europe…","""Administration…","""diplomat""","""5.Contemporary…","""Missing""",25.141882,230781.0,"""Russia""","""Europe""",44.514721,null,48.708611,null,"""Vasily Nebenzy…","""Russian diplom…","[""United Nations""]","[""Moscow State Institute of International Relations""]",[],[],"""02""",1,0,1,2,1,2,0
"""Q1000061""",1940,null,"""Male""","""Leadership""","""Valentyn_Symon…","""Eastern Europe…","""Politics""","""politician""","""5.Contemporary…","""Missing""",23.340572,219961.0,"""Ukraine""","""Europe""",30.73262,null,46.47747,null,"""Valentyn Symon…","""Ukrainian stat…","[""State University of Trade and Economics""]","[""Odesa State Academy of Civil Engineering and Architecture""]",[],[],"""07""",1,0,1,1,1,2,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Q999994""",1988,null,"""Male""","""Sports/Games""","""Mitja_Mežnar""","""Southern Europ…","""Sports/Games""","""ski""","""5.Contemporary…","""Missing""",21.543238,350041.5,"""Slovenia""","""Europe""",14.35561,null,46.238869,null,"""Mitja Mežnar""","""Slovenian ski …",[],[],[],[],"""06""",1,0,0,1,0,1,0
"""Q999995""",1987,null,"""Male""","""Sports/Games""","""Martin_Cikl""","""Western Europe…","""Sports/Games""","""ski""","""5.Contemporary…","""Missing""",21.71133,339942.5,"""Czech_Republic…","""Europe""",14.618354,null,50.911613,null,"""Martin Cikl""","""Czech ski jump…",[],[],[],[],"""08""",1,0,0,1,0,1,0
"""Q999997""",1984,null,"""Male""","""Sports/Games""","""Vincent_Descom…","""Western Europe…","""Sports/Games""","""ski""","""5.Contemporary…","""Missing""",24.453411,154890.0,"""France""","""Europe""",6.868889,null,45.922222,null,"""Vincent Descom…","""French ski jum…",[],[],[],[],"""01""",1,0,0,1,0,2,0
